<a href="https://colab.research.google.com/github/Ruheena-S/Hierarchical-classification-Loss-Functions-in-Image-Classification/blob/main/OVA_ResNet18_CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip3 install torch_optimizer torchmetrics
!nvidia-smi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mon Mar 13 10:01:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    34W /  70W |   2993MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-------------

In [19]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_optimizer import Ranger
from torchvision import datasets
from torchvision import transforms,models
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchmetrics import Accuracy
from torch.optim.lr_scheduler import CyclicLR

torch.manual_seed(43)

In [20]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:

def target_trans(target):
  y = -torch.ones(100)
  y[target] = 1

  return y,target 

In [22]:
# Define transforms for data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])
])

dataset = datasets.CIFAR100(root='data/', download=True, transform=transform_train,target_transform=target_trans)
test_dataset = datasets.CIFAR100(root='data/', train=False, transform=transform_test,target_transform=target_trans)


val_size = 5000
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
batch_size=256

train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(val_ds, batch_size, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size, num_workers=4)

Files already downloaded and verified


In [23]:
class Hinge_Loss(torch.nn.Module):
    
    def __init__(self):
        super(Hinge_Loss,self).__init__()
        
    def forward(self,y_p,y_t):

        temp = 1 - y_p * y_t
        
        zero = torch.zeros(y_p.size()[0],y_p.size()[1])
        zero = zero.cuda()
        clamp = torch.max(temp,zero)
    
        total_loss = torch.sum(clamp)/y_p.size()[0]
        return total_loss

In [24]:
num_classes = 100
num_epochs = 10


In [25]:
# Load pre-trained ResNet18 model
resnet18 = models.resnet18(pretrained=True)
resnet18.conv1 = nn.Conv2d(3, 64, kernel_size = (3,3), padding = (1, 1), bias = False)
resnet18.maxpool = nn.Identity()

# Freeze all layers except for the last one
for param in resnet18.parameters():
    param.requires_grad = True
resnet18.fc = nn.Linear(512, 100)

resnet18.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), p

In [26]:
# Define loss function and optimizer

criterion = Hinge_Loss().to(device)
accuracy = Accuracy(task="multiclass",num_classes=num_classes).to(device)
optimizer = Ranger(resnet18.parameters(), lr=1e-3, weight_decay=1e-4) 
scheduler = CyclicLR(optimizer, base_lr=1e-6, max_lr=1e-3, step_size_up=len(train_loader)//2, cycle_momentum=False)

# optimizer = optim.SGD(resnet18.fc.parameters(), lr=0.001, momentum=0.9)
#optimizer = Ranger(resnet18.parameters(), lr=0.001, weight_decay=0.005) 
#optimizer = optim.Adam(resnet18.parameters(), lr=0.001)


In [27]:

def get_accuracy(output, target):

  batch_size = target.size(0)
  correct=0

  pred = output.max(dim=1)[1]

  correct = pred==target

  acc = correct.float().sum(0)

  return acc/batch_size


In [30]:
num_epochs = 40
total_train_step = len(train_loader)
#print(total_train_step)
total_val_step=len(valid_loader)
BEST_VAL_METRIC = 0
BEST_MODEL = None

for epoch in range(1, num_epochs+1):

    train_loss=0
    train_acc=0.0
    resnet18.train()

    for i, (images, target) in enumerate(train_loader, 1):

        y_trans = target[0]
        y_true = target[1]

        # Move tensors to the configured device
        images = images.to(device)
        y_true = y_true.to(device)
        y_trans = y_trans.to(device)
        
        # Forward pass
        outputs = resnet18(images)
        loss = criterion(outputs, y_trans)

        train_loss += loss
        #train_acc += get_accuracy(outputs, y_true)
        train_acc += accuracy(outputs, y_true)
        
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    print(f'Epoch [{epoch}/{num_epochs}] - Loss: {(train_loss/total_train_step):.4f}, Accuracy: {(train_acc/total_train_step):.4f}')

    resnet18.eval() 
    # Validation
    with torch.no_grad():
        val_acc = 0
        val_loss=0
        for i, (images, target) in enumerate(valid_loader, 1):

            y_trans = target[0]
            y_true = target[1]

            # Move tensors to the configured device
            images = images.to(device)
            y_true = y_true.to(device)
            y_trans = y_trans.to(device)

            outputs = resnet18(images)
            val_loss += criterion(outputs, y_trans)
            #val_acc += get_accuracy(outputs, y_true)
            val_acc += accuracy(outputs, y_true)

    if val_acc/total_val_step > BEST_VAL_METRIC:
        BEST_VAL_METRIC = val_acc/total_val_step
        BEST_MODEL = resnet18.state_dict() 

    print(f'Accuracy of the network on the 5000 validation images: {(val_acc/total_val_step):.4f}, loss: {(val_loss/total_val_step):.4f}') 

Epoch [1/40] - Loss: 2.0191, Accuracy: 0.2171
Accuracy of the network on the 5000 validation images: 0.2533, loss: 2.0108
Epoch [2/40] - Loss: 2.0305, Accuracy: 0.2229
Accuracy of the network on the 5000 validation images: 0.2552, loss: 2.0249
Epoch [3/40] - Loss: 2.0147, Accuracy: 0.2350
Accuracy of the network on the 5000 validation images: 0.2723, loss: 1.9962
Epoch [4/40] - Loss: 2.0062, Accuracy: 0.2361
Accuracy of the network on the 5000 validation images: 0.2795, loss: 1.9866
Epoch [5/40] - Loss: 1.9936, Accuracy: 0.2628
Accuracy of the network on the 5000 validation images: 0.3036, loss: 1.9776
Epoch [6/40] - Loss: 1.9827, Accuracy: 0.2793
Accuracy of the network on the 5000 validation images: 0.3209, loss: 1.9573
Epoch [7/40] - Loss: 1.9423, Accuracy: 0.3072
Accuracy of the network on the 5000 validation images: 0.3494, loss: 1.9215
Epoch [8/40] - Loss: 1.9145, Accuracy: 0.3284
Accuracy of the network on the 5000 validation images: 0.3780, loss: 1.8980
Epoch [9/40] - Loss: 1.8

In [31]:
#Testing
resnet18.load_state_dict(BEST_MODEL)

total_test_step=len(test_loader)

with torch.no_grad():
    test_acc=0
    test_loss=0

    for i, (images, target) in enumerate(test_loader, 1):
        
        y_trans = target[0]
        y_true = target[1]
        
        images = images.to(device)
        y_true = y_true.to(device)
        y_trans = y_trans.to(device)

        # Forward pass
        outputs = resnet18(images)
        
        # Loss
        test_loss += criterion(outputs,y_trans)
        test_acc += accuracy(outputs, y_true)

    print(f'Accuracy of the network on test images: {(test_acc/total_test_step):.4f}, loss: {(test_loss/total_test_step):.4f}')

Accuracy of the network on test images: 0.7336, loss: 1.2873
